# Import Libraries

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip install tensorflow==2.12

In [ ]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

In [ ]:
import os
import glob
import time
import timeit
import pandas as pd
import numpy as np
from scipy import signal
# from tqdm import tqdm
from tqdm.notebook import tqdm
from pathlib import Path
from math import sqrt
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Data File Count & Duration

In [ ]:
# Define CSI_data gdrive URL
data_url = '/content/drive/MyDrive/CSI_Test_Data/with_person'

raw_data = []
raw_data_duration = []

# Get the list of CSV files
file_list = glob.glob(data_url + '/*.csv')

print("Total File Count:", len(file_list))
print("==================================================================")

# Process random files with
print("Processing random test data files:")
for file_name in tqdm(file_list, desc="random test data", unit="file"):
  try:
      raw_data.append(file_name)
      extract_duration_data = pd.read_csv(file_name, low_memory=False)
      extract_duration_data = extract_duration_data['real_timestamp'].values
      extract_duration_data = (extract_duration_data[-1] - extract_duration_data[0]) / 60
      raw_data_duration.append(extract_duration_data)
  except Exception as e:
    print(f"Error processing file {file_name}: {e}")
  except ValueError as ve:
    print(f"ValueError encountered: {ve}")
    continue

print("==================================================================")
print("Found {} random test data files.".format(len(raw_data)))
print("With a total duration of {} minutes.".format(sum(raw_data_duration)))
print("==================================================================")

# Filter Design


In [ ]:
def lowpass(csi_vec: np.array, cutoff: float, fs: float, order: int) -> np.array:
    nyq = 0.5*fs
    normal_cutoff = cutoff/nyq
    b, a = signal.butter(order, normal_cutoff, btype="low", analog=False)
    return signal.filtfilt(b, a, csi_vec)

def running_mean(x: np.array, N: int) -> np.array:
    return pd.Series(x).rolling(window=N, min_periods=1, center=True).mean().to_numpy()

#  Read File Function

In [ ]:
def read_csv(data_url, window_size, step_size):

    scaler = StandardScaler()
    data_test_windowed = []

    data_test = []

    file_list = glob.glob(data_url + '/*.csv')

    # File processing progress
    for file_name in tqdm(file_list, desc="Processing Files", unit="file"):
        print(file_name)
        TestData = pd.read_csv(file_name, low_memory=False)
        TestData = TestData['CSI_DATA'].values
        TestData = TestData[500:-500]

        # Data processing progress
        for i in tqdm(range(TestData.shape[0]), desc="Processing random test data", leave=False):
            try:
                st = TestData[i]
                st = st[1:-2]
                data_array = [int(s) for s in st.split(' ')]
                data_array_mag = []
                for k in range(0, 128, 2):
                    data_array_mag.append(sqrt(data_array[k]**2 + data_array[k+1]**2))
                data_test.append(data_array_mag)
            except Exception as e:
                #print(f"Error encountered at row {i}: {TestData[i]}")
                print(f"Error encountered: {e}")
                continue
            except ValueError as ve:
                #print(f"ValueError encountered at row {i}: {TestData[i]}")
                print(f"ValueError encountered: {ve}")
                continue

    data_test = np.array(data_test)

    # Window processing progress for test data
    for start in tqdm(range(0, data_test.shape[0] - window_size, step_size), desc="Creating random test-data Windows", unit="window"):
        end = start + window_size
        data_test_window = np.empty((0, 64))
        for j in range(start, end):
            data_array_mag = data_test[j]
            data_test_window = np.append(data_test_window, np.array([data_array_mag]), axis=0)
        for i in range(0, 64):
            data_test_window[:, i] = lowpass(data_test_window[:, i], 30, 170, 5)
            data_test_window[:, i] = running_mean(data_test_window[:, i], 10)
        data_test_window = scaler.fit_transform(data_test_window)
        data_test_windowed.append(data_test_window)

    data_test = data_test_windowed
    return np.array(data_test)


# Data Extraction

In [ ]:
window_size = 300  # Adjust Window_Size
step_size = 50

save_url_test = 'data_test_window_size={window}_step_size={step}.npy'.format(window = window_size, step = step_size)
save_url_test = Path(save_url_test)

if (save_url_test.is_file() and save_url_test.is_file()):
  data_np = np.load(save_url_test)
else:
  file1 = 'data_test_window_size={window}_step_size={step}.npy'.format(window = window_size, step = step_size)

  data_test = read_csv(data_url, window_size=window_size, step_size=step_size)
  np.save(file1,data_test)

In [ ]:
print(data_test.shape)

#Model Load & Predict

In [ ]:
import keras

from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
import tensorflow as tf
from tensorflow.keras.utils import to_categorical



from keras.models import load_model
model = load_model('/content/drive/MyDrive/Trained_Model/lstm_model_300.h5')
model.summary()

In [ ]:
import time
start_time = time.time()
predict = model.predict(data_test)
end_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))
# print(predict)
# print(predict.shape)

In [ ]:
# CPU processing Time
start_time = time.process_time()
predict = model.predict(data_test)
end_time = time.process_time()
print("Testing time per block: %.2f ms" %((end_time-start_time)*1000/data_test.shape[0]))

In [ ]:
pred_thr = np.where(predict[:,0] >0.5, 1,0)
pred_thr = np.expand_dims(pred_thr, axis=-1)
# print(pred_thr)
print(pred_thr.shape)
count = np.array(pred_thr)
unique, counts = np.unique(count, return_counts=True)
dict(zip(unique, counts))
print("With Person #blocks (1):", counts[1])
print("No Person #blocks (0):", counts[0])

### **The End**
